In [1]:
!pip install eli5

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
import eli5
from eli5.sklearn import PermutationImportance
from nltk.corpus import stopwords
from string import punctuation

In [4]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [5]:
data = pd.read_csv('/gdrive/My Drive/exam/jigsaw-toxic-comment-train.csv.zip')

## 1. Описательный анализ данных с привязкой к целевой переменной (toxic) и 2 другим дополнительным колонкам (как минимум 5 статистик на каждую группу) - 1.5 балл

In [6]:
data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [7]:
data.shape

(223549, 8)

In [8]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [9]:
stops = stopwords.words('english')

In [10]:
data['text_len'] = data['comment_text'].apply(len) #длина текста
data['text_len_tokens'] = data['comment_text'].apply(lambda x: len(x.split())) #количество токенов
data['number_of_caps'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch.isupper()])) #количество слов с заглавной буквы
data['number_of_nonalphanum'] = data['comment_text'].apply(lambda x: len([ch for ch in x if not ch.isalnum()])) #количество  не буквенно-цифровых символов
data['number_of_lows'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch.islower()])) #количество строчных букв
data['number_of_digits'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch.isdigit()])) #количество цифр
data['number_of_ascii'] = data['comment_text'].apply(lambda x: len([ch for ch in x if is_ascii(ch)])) #количество ascii символов
data['number_of_nonascii'] = data['comment_text'].apply(lambda x: len([ch for ch in x if not is_ascii(ch)])) #количество non-ascii символов
data['number_of_punctuation'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch in punctuation])) #количество знаков пунктуации
data['number_of_stops'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch in stops])) #количество стоп-слов

In [11]:
data.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,264,43,17,52,186,9,264,0,10,84
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,112,17,8,29,65,10,112,0,12,29
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,233,42,4,47,182,0,233,0,6,99


### Toxic/Non-toxic

In [13]:
data.groupby('toxic')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
toxic,,,,,,,,,,
0,402.691178,68.415161,14.484189,89.072456,294.922009,3.248698,400.972142,1.719036,16.974704,141.799609
1,280.604097,48.573466,44.689675,65.266741,168.618921,1.995885,280.409512,0.194585,14.257202,80.432941


1. Средняя длина нетоксичного комментария выше, чем у токсичного. 
2. Среднее количество слов у нетоксичных текстов также больше. 
3. Среднее количество используемых заглавных букв у токсичных сообщений в несколько рах выше, чем у нетоксичных.
4. Среднее количество  не буквенно-цифровых символов в нетоксичных комментариях выше.
5. Количество строчных букв в токсичных комментариях меньше, чем в нетоксичных.
6. Цифр в нетоксичных комментариях также в среднем больше. 
7. Ascii-символов в нетоксичных комментариях больше.
8. Не-ascii символов в целом по комментариям практически нет, но если они встречаются, то чаще в нетоксичных комментариях.
9. Оба типа комментариев имеют практически одинаковое среднее количество знаков препинания. В нетоксичных комментариях их чуть больше. 
10. Используемых стоп-слов в нетоксичных комментариях гораздо больше, чем в токсичных.

### Severe_toxic/Non-severe_toxic

In [14]:
data.groupby('severe_toxic')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
severe_toxic,,,,,,,,,,
0,389.999386,66.358270,15.988131,86.500630,283.491852,3.136258,388.413228,1.586158,16.591014,136.307514
1,505.456677,84.462793,173.841998,120.072375,209.248726,2.293068,505.345566,0.111111,30.690112,93.232926


1. Средняя длина сильно токсичных комментариев гораздо больше. 
2. Среднее количество слов в сильно токсичных комментариях выше.
3. Количество заглавных букв в сильно токсичных текстах во много раз превышает количество в обычных комментариях.
4. Среднее количество не буквенно-цифровых символов в токсичных комментариях выше.
5. Строчных букв в обычных комментариях больше. 
6. В обоих типах комментариев цифр мало, но в обычных комментариях в среднем чуть больше.
7. Ascii-символов больше в сильно токсичных комментариях.
8. Не-ascii символы практически отсутствуют в обоих типах. В обычных чуть больше.
9. Количество знаков пунктуации в среднем почти в 2 раза больше в токсичных комментариях.
10. Количество стоп-слов в обычных комментариях в среднем больше. 

### Obscene/Non-obscene

In [15]:
data.groupby('obscene')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
obscene,,,,,,,,,,
0,397.332876,67.559707,15.299590,88.032052,289.883993,3.193029,395.678353,1.654523,16.850087,139.417163
1,280.951730,48.362109,53.490033,65.257743,160.178748,2.011367,280.794481,0.157249,14.358072,75.193822


### Описание TBA

### Threat/Non-threat

In [16]:
data.groupby('threat')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
threat,,,,,,,,,,
0,391.241807,66.547128,17.149430,86.796312,283.283766,3.134856,389.665157,1.576649,16.686637,136.141282
1,316.910015,56.825835,89.866473,86.460087,139.383164,1.188679,316.448476,0.461538,25.809869,67.416546


### Описание TBA

### Insult/Non-insult

In [17]:
data.groupby('insult')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
insult,,,,,,,,,,
0,397.340234,67.544536,15.573931,88.053302,289.597522,3.194615,395.690669,1.649565,16.879710,139.233245
1,272.206476,47.227176,51.163393,63.174452,155.965057,1.894197,272.066879,0.139597,13.617569,73.897381


### Описание TBA

### Identity_hate/Non-identity_hate

In [18]:
data.groupby('identity_hate')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
identity_hate,,,,,,,,,,
0,391.744793,66.651428,16.774599,86.960259,283.989184,3.137812,390.158234,1.586559,16.740187,136.492779
1,314.438829,52.473784,80.022201,69.538498,162.665092,2.192253,314.261691,0.177137,14.054795,77.008503


### Описание TBA

### Описательный анализ с привязкой к другим колонкам

In [19]:
data.groupby(['toxic', 'identity_hate'])['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


text_len  ... number_of_stops
                           mean  ...            mean
toxic identity_hate              ...                
0     0              402.695179  ...      141.799127
      1              396.065574  ...      142.598361
1     0              277.636340  ...       81.197999
      1              309.447118  ...       72.997494

[4 rows x 10 columns]

### 2. Бейзлайн модель из sklearn (векторайзер + модель) с отбором признаков (через l1 регуляризацию, на глаз через анализ важных параметров или через permutation importance) - 2 балл

In [35]:
X = data['comment_text']
y = data['toxic']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [37]:
count_vect = CountVectorizer()

In [38]:
X_train = count_vect.fit_transform(X_train)
X_train.shape

(149777, 207901)

In [39]:
X_test = count_vect.transform(X_test)
X_test.shape

(73772, 207901)

In [40]:
clf = SGDClassifier().fit(X_train, y_train)

In [41]:
y_pred = clf.predict(X_test)

In [42]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     66765
           1       0.81      0.60      0.69      7007

    accuracy                           0.95     73772
   macro avg       0.89      0.79      0.83     73772
weighted avg       0.95      0.95      0.95     73772



In [43]:
eli5.formatters.as_dataframe.explain_weights_df(clf)


,target,feature,weight
0,1,x70089,2.186657
1,1,x84835,2.092285
2,1,x64497,2.032440
3,1,x20704,2.011724
4,1,x84845,1.852904
...,...,...,...
70655,1,<BIAS>,-1.181929
70656,1,x23850,-1.284373
70657,1,x69507,-1.394857
70658,1,x166952,-1.629635


In [47]:
sum(sum(clf.coef_ != 0))


70659

In [44]:
top_features = [int(i[1:]) for i in eli5.formatters.as_dataframe.explain_weights_df(clf).feature if 'BIAS' not in i]

In [46]:
len(top_features)

70659

In [48]:
X_train_eli5 = X_train[:,top_features]
X_test_eli5 = X_test[:,top_features]

In [49]:
eli5_model = SGDClassifier().fit(X_train_eli5, y_train)

In [52]:
y_pred = eli5_model.predict(X_test_eli5)

In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     66765
           1       0.81      0.61      0.70      7007

    accuracy                           0.95     73772
   macro avg       0.89      0.80      0.83     73772
weighted avg       0.95      0.95      0.95     73772



### 3. 